<a href="https://colab.research.google.com/github/RyosukeHanaoka/TechTeacher/blob/main/dicomFileToFinalPng.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#必要なモジュールのインストール

In [1]:
!pip install pydicom
!pip install pylibjpeg
!pip install python-gdcm
!pip install pylibjpeg-libjpeg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 76.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 3.5 MB/s eta 0:00:00


In [2]:
# まず、使用するライブラリを呼び出す。
from google.colab import drive
import os
import shutil
import pydicom
import cv2
import numpy as np

In [3]:
# Google Driveを呼び出す。
drive.mount('/content/drive')

Mounted at /content/drive


#作業に使用するxpFilesのコピーを作成する。

In [5]:
# xpFilesのコピーを作成する。
shutil.copytree('/content/drive/MyDrive/xpFiles','/content/drive/MyDrive/xpFiles_copy')

'/content/drive/MyDrive/xpFiles_copy'

#directory名に含まれている患者idをDICOMファイルのファイル名の先頭に付加する。

In [6]:
for root, dirs, files in os.walk('/content/drive/MyDrive/xpFiles_copy'):# '/content/drive/MyDrive/xpFiles_copy'の中のroot、directory、fileを走査する。
    for file in files:
        # 変数patient_idを定義
        patient_id = os.path.dirname(root)[40:46]
        # 変数patient_filename_taleを定義
        patient_filename_tale = os.path.basename(root)
        # 変数new_filenameを定義
        file_extension = os.path.splitext(file)[1]  # 元のファイルの拡張子を取得
        new_filename = patient_id+'_'+ patient_filename_tale+".dcm"
        os.rename(os.path.join(root, file), os.path.join(root, new_filename))

###ファイルの損傷に備えてxpFiles_copyはバックアップとして保存、xpFiles_copy_of_copyを作成

In [7]:
# xpFiles_copyのコピーを作成する。
shutil.copytree('/content/drive/MyDrive/xpFiles_copy','/content/drive/MyDrive/xpFiles_copy_of_copy')

'/content/drive/MyDrive/xpFiles_copy_of_copy'

#DICOMの'BodyPartExamined'が'HAND'のファイルのみ抽出、'xpFiles_Hand'に保存

In [8]:
os.makedirs('/content/drive/MyDrive/xpFiles_Hand', exist_ok=True)

for root, dirs, files in os.walk('/content/drive/MyDrive/xpFiles_copy_of_copy'):
    for file in files:
        dcmfile_path = os.path.join(root, file)
        ds = pydicom.dcmread(dcmfile_path, force=True)

        if hasattr(ds, 'BodyPartExamined'):
            body_part = ds.BodyPartExamined
            if body_part == 'HAND':
                shutil.move(dcmfile_path, '/content/drive/MyDrive/xpFiles_Hand')

In [15]:
#xpFiles_Handのコピーを作成する。
shutil.copytree('/content/drive/MyDrive/xpFiles_Hand','/content/drive/MyDrive/xpFiles_Hand_copy')

'/content/drive/MyDrive/xpFiles_Hand_copy'

#DICOMをPNGに変換する。

In [4]:
import numpy as np
import pydicom
import pylibjpeg
from PIL import Image
import os

In [6]:
import numpy as np
import cv2
import pydicom
import os

def dicom_to_png(dcmfile_path, png_directory):
    try:
        ds = pydicom.dcmread(dcmfile_path)

        rescale_slope = ds.get("RescaleSlope", 1)
        rescale_intercept = ds.get("RescaleIntercept", 0)

        img = ds.pixel_array * rescale_slope + rescale_intercept

        # 画像を0-255の範囲に正規化
        img = np.interp(img, (img.min(), img.max()), (0, 255))

        pngfile_path = os.path.join(png_directory, os.path.basename(dcmfile_path).replace('.dcm', '.png'))
        cv2.imwrite(pngfile_path, img.astype(np.uint8))

    except Exception as e:  # すべての例外をキャッチ
        print(f"An error occurred while processing {dcmfile_path}: {str(e)}")

png_directory = '/content/drive/MyDrive/xpFilesFinal_png'
os.makedirs(png_directory, exist_ok=True)

for root, dirs, files in os.walk('/content/drive/MyDrive/xpFiles_Hand_copy'):
    for file in files:
        dcmfile_path = os.path.join(root, file)
        dicom_to_png(dcmfile_path, png_directory)

In [7]:
shutil.copytree('/content/drive/MyDrive/xpFilesFinal_png','/content/drive/MyDrive/backUpofXpFilesFinal_png')

'/content/drive/MyDrive/backUpofXpFilesFinal_png'